In [ ]:
import geopandas as gpd
import pandas as pd
import contextily as ctx
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
from matplotlib import colors

# PART 1: DATA PREPROCESSING

## 1. Dataset: Surface Temperature

In [ ]:
gdf_st_clean = gpd.read_file('Surface_temperature-shp.zip')
gdf_st_clean

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.2)
gdf_st_clean.to_crs(epsg=3857).plot(ax=ax, markersize=10, alpha=0.6, column='Teplota', cmap='plasma', legend=True, cax=cax)
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=12)
cb_ax.set_ylabel("\nTeplota [°C]", fontsize=15)

ctx.add_basemap(ax, crs=gdf_st_clean.to_crs(epsg=3857).crs.to_string(), source=ctx.providers.Stamen.Terrain)

fig.savefig('01_Surface_Temperature_clean.png', bbox_inches='tight', dpi=250)

In [ ]:
# GeoDataFrame Surface Temperature
gdf_st = gdf_st_clean.copy()
gdf_st = gdf_st.to_crs(epsg=3857)
gdf_st = gdf_st.set_geometry(gdf_st.centroid)
gdf_st

## 2. Dataset: Building height

In [ ]:
gdf_bh_clean = gpd.read_file('Building_heights-shp.zip')
gdf_bh_clean

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()

gdf_bh_clean.to_crs(epsg=3857).plot(ax=ax)
ctx.add_basemap(ax, crs=gdf_st_clean.to_crs(epsg=3857).crs.to_string(), source=ctx.providers.Stamen.Terrain)

fig.savefig('02_Building_Height_clean.png', bbox_inches='tight', dpi=250)

In [ ]:
gdf_bh = gdf_bh_clean.copy().to_crs(epsg=5514)  # Krovak projection to calculate area
gdf_bh = gdf_bh[gdf_bh['vyska_komp'].notna()]   # drop NaN values
gdf_bh['area'] = gdf_bh.area                    # calculate area in m2
gdf_bh = gdf_bh.to_crs(epsg=3857)               # back to Mercator projection
gdf_bh

In [ ]:
x_min, y_min, x_max, y_max = gdf_bh.total_bounds

# Split the bh map into NxN grid
N = 100
x_side = (x_max - x_min)/N
y_side = (y_max - y_min)/N

grid = []
for j in range(N):
    for i in range(N):
        x_point_list = [x_min + i*x_side, x_min + (i+1)*x_side, x_min + (i+1)*x_side, x_min + i*x_side, x_min + i*x_side]
        y_point_list = [y_min + j*y_side, y_min + j*y_side, y_min + (j+1)*y_side, y_min + (j+1)*y_side, y_min + j*y_side]
        grid.append(Polygon(zip(x_point_list, y_point_list)))

gdf_grid = gpd.GeoDataFrame(grid, columns=['geometry'])
gdf_grid = gdf_grid.set_crs('epsg:3857')
gdf_grid

In [ ]:
# Sides of 1 tile in grid
print("x dimension of 1 tile: %.2f m" %x_side)
print("y dimension of 1 tile: %.2f m" %y_side)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()
gdf_grid["geometry"].boundary.plot(color='k', ax=ax, linewidth=0.5, alpha=0.4)
gdf_bh.to_crs(epsg=3857).plot(ax=ax)
ctx.add_basemap(ax, crs=gdf_bh.crs.to_string(), source=ctx.providers.Stamen.Terrain)

fig.savefig('03_Building_Height_grid.png', bbox_inches='tight', dpi=250)

## Create/Load dataframe of intersections between grid and building height
To create dataframe of intersections uncomment commented lines below. It takes approx 180 min.
Otherwise dataframe will be loaded from pickled file.

In [ ]:
# gdf_bh_grid = gpd.GeoDataFrame(columns=['geometry','rectangle_ID', 'building_ID'])

# for ID, rectangle in enumerate(gdf_grid.geometry):                          # for each tile in grid
#     df_tmp = gpd.GeoDataFrame(columns=['geometry','rectangle_ID'])          # prepare empty GeoDataFrame
#     df_tmp.geometry = gdf_bh.intersection(rectangle)                        # get intersection of all buildings with current tile
#     df_tmp = df_tmp[~df_tmp.is_empty]                                       # drop all empty intersections
#     df_tmp["rectangle_ID"] = ID                                             # mark tile ID
#     df_tmp = df_tmp.reset_index().rename(columns={'index': 'building_ID'})
#     all_df = all_df.append(df_tmp)
#     if ID%1000==0:
#         print(ID)
# gdf_bh_grid.to_pickle("./gdf_bh_grid.pkl")

# Load dataframe already created
gdf_bh_grid = pd.read_pickle("./gdf_bh_grid.pkl")
gdf_bh_grid = gdf_bh_grid.set_crs('epsg:3857')
gdf_bh_grid['area_building_in'] = gdf_bh_grid.to_crs(epsg=5514).area    # Krovak to calculate area of parts of buildings inside squares
gdf_bh_grid = gdf_bh_grid.to_crs(epsg=3857)                             # Back to Mercator for nicer visualization
gdf_bh_grid

In [ ]:
gdf_bh_grid = gdf_bh_grid.merge(gdf_bh, left_on='building_ID', right_index=True)
gdf_bh_grid = gdf_bh_grid.assign(typ_budovy=gdf_bh_grid["zpusob_vyu"].values) # make copy of values from "zpusob_vyu" column into "building_type" because they will be used twice
gdf_bh_grid

In [ ]:
df_bh_grid = gdf_bh_grid.pivot_table(index='rectangle_ID', columns=['typ_budovy'],
                                values=['zpusob_vyu','vyska_komp','area_building_in', "OBJECTID_1"],
                                aggfunc={"zpusob_vyu":"count","vyska_komp":np.mean, "area_building_in":np.sum, "OBJECTID_1":np.mean}
                                ).rename(columns={'zpusob_vyu': 'cnt', 'vyska_komp': 'avg_height', 'area_building_in': 'b_area_in_rect', "OBJECTID_1":"b_area_ratio"})
df_bh_grid['b_area_ratio'] = np.nan
df_bh_grid['b_area_ratio'] = df_bh_grid['b_area_in_rect'].div(df_bh_grid['b_area_in_rect'].sum(axis=1), axis=0)
df_bh_grid

### Height of tiles
Calculate Weighted Mean of buildings height according to the area they occupy in given tile

In [ ]:
gdf_grid_extended = gdf_grid.loc[df_bh_grid.index.tolist(), :].copy()
gdf_grid_extended["WM_Height"] = np.nan
for idx in gdf_grid_extended.index:
    area = df_bh_grid.loc[idx, 'b_area_in_rect'].loc[~df_bh_grid.loc[idx, 'b_area_in_rect'].isnull()].values
    height = df_bh_grid.loc[idx, 'avg_height'].loc[~df_bh_grid.loc[idx, 'avg_height'].isnull()].values
    weighted_mean = np.dot(area,height) / np.sum(area)
    gdf_grid_extended.loc[idx, 'WM_Height'] = weighted_mean
gdf_grid_extended

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 15))
ax.set_axis_off()

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.2)
gdf_grid_extended.plot(ax=ax, alpha=0.8, column='WM_Height', cmap='viridis', legend=True, cax=cax)
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=12)
cb_ax.set_ylabel("\nVýška [m]", fontsize=15)

ctx.add_basemap(ax, crs=gdf_st_clean.to_crs(epsg=3857).crs.to_string(), source=ctx.providers.Stamen.Terrain)
plt.tight_layout()

fig.savefig('04_Building_Height.png', bbox_inches='tight', dpi=250)

### Temperature of tiles
Take temperature points that are inside building height dataset with 10% border, to not interpolate the whole temperature map when buildings are only in some areas

In [ ]:
# Outer boundary of grid
gdf_1Square = gpd.GeoDataFrame()

x_point_list = [x_min, x_min + (N+1)*x_side, x_min + (N+1)*x_side, x_min, x_min]
y_point_list = [y_min, y_min, y_min + (N+1)*y_side, y_min + (N+1)*y_side, y_min]

gdf_1Square = gpd.GeoDataFrame(geometry=[Polygon(zip(x_point_list, y_point_list))])
gdf_1Square = gdf_1Square.set_crs('epsg:3857')
gdf_1Square

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()
gdf_bh.plot(ax=ax)
gdf_1Square["geometry"].boundary.plot(color='k', ax=ax, linewidth=0.5)
gdf_1Square.buffer(0.1*(x_max-x_min)).boundary.plot(color='k', ax=ax, linewidth=0.5)
ctx.add_basemap(ax, crs=gdf_1Square.crs.to_string(), alpha=0.8)

In [ ]:
# Get temperature of points inside buffer
gdf_TiB = gpd.GeoDataFrame()
gdf_TiB.geometry = gdf_st.intersection(gdf_1Square.buffer(0.1*(x_max-x_min)).geometry.iloc[0])
gdf_TiB = gdf_TiB.set_crs("epsg:3857")

gdf_TiB = gdf_TiB[~gdf_TiB.geometry.is_empty]                                   # Remove empty points
gdf_TiB = gdf_TiB.merge(gdf_st["Teplota"], left_index=True, right_index=True)   # Add temperature values
gdf_TiB

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 15))
ax.set_axis_off()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.2)

gdf_grid["geometry"].boundary.plot(color='k', ax=ax, linewidth=0.5, alpha=0.2)
gdf_1Square.buffer(0.1*(x_max-x_min)).boundary.plot(color='k', ax=ax, alpha=0.2, linewidth=0.5)
gdf_TiB.plot(ax=ax, markersize=10, alpha=0.7, column='Teplota', cmap='plasma', legend=True, cax=cax)
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=12)
cb_ax.set_ylabel("\nTeplota [°C]", fontsize=15)

ctx.add_basemap(ax, crs=gdf_st_clean.to_crs(epsg=3857).crs.to_string(), source=ctx.providers.Stamen.Terrain)
plt.tight_layout()

fig.savefig('05_Surface_Temperature_centroids.png', bbox_inches='tight', dpi=250)

In [ ]:
gdf_grid["Temperature"] = np.nan

points = np.vstack((gdf_TiB.centroid.x, gdf_TiB.centroid.y)).T
values = gdf_TiB.Teplota.values

grid_x, grid_y = np.meshgrid(gdf_grid.loc[df_bh_grid.index.tolist(),:].centroid.x.values, gdf_grid.loc[df_bh_grid.index.tolist(),:].centroid.y.values)
grid_z = griddata(points, values, (grid_x, grid_y), method='linear')
gdf_grid.loc[df_bh_grid.index.tolist(), "Temperature"] = np.diagonal(grid_z)
gdf_grid

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 15))
ax.set_axis_off()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.2)

gdf_grid.plot(ax=ax, markersize=10, alpha=0.7, column='Temperature', cmap='plasma', legend=True, cax=cax)
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=12)
cb_ax.set_ylabel("\nTeplota [°C]", fontsize=15)

ctx.add_basemap(ax, crs=gdf_st_clean.to_crs(epsg=3857).crs.to_string(), source=ctx.providers.Stamen.Terrain)
fig.savefig('06_Surafe_Temperature_grid.png', bbox_inches='tight', dpi=250)

Add temperature to the df

In [ ]:
df_bh_grid.loc[:, "Temperature"] = gdf_grid.loc[df_bh_grid.index.tolist(), "Temperature"]
df_bh_grid

# PART 2: ANALYSIS

A "quick look" at Building Height dataset to better understand and see the count of each building type, average height, average area and total area covered

In [ ]:
gdf_bh['area2'] = gdf_bh['area']
gdf_bh.groupby('zpusob_vyu').agg({"OBJECTID_1":"count","vyska_komp":np.mean, "area":np.mean, "area2":np.sum}
                                ).rename(columns={'OBJECTID_1': 'cnt', 'vyska_komp': 'avg_height', 'area': 'avg_area', 'area2': 'sum_area'}
                                ).sort_values(by='cnt', ascending=False)

## Split tiles into 3 groups based on temperature
### Tiles with:
- Temperature below 21°C
- Temperature between 21°C and 25°C
- Temperature above 25°C

In [ ]:
labels = ['nad 25°C', '21°C ÷ 25°C', 'pod 21°C']
values = [df_bh_grid[df_bh_grid["Temperature"] > 25 ]['b_area_ratio'].sum(axis=0).sum(axis=0) * 100 / df_bh_grid['b_area_ratio'].sum(axis=0).sum(axis=0),
        df_bh_grid[(df_bh_grid["Temperature"] <= 25) & (df_bh_grid["Temperature"] >= 21)]['b_area_ratio'].sum(axis=0).sum(axis=0) * 100 / df_bh_grid['b_area_ratio'].sum(axis=0).sum(axis=0),
        df_bh_grid[df_bh_grid["Temperature"] < 21 ]['b_area_ratio'].sum(axis=0).sum(axis=0) * 100 / df_bh_grid['b_area_ratio'].sum(axis=0).sum(axis=0)]
explode = (0.15,0.05,0.1)
colors = [sns.color_palette('plasma')[i] for i in [5,2,0]]

fig1, ax1 = plt.subplots(figsize=(5,3))
patches, labels, pct_texts = ax1.pie(values, labels=labels, labeldistance=1.05, autopct='%1.1f%%', pctdistance=0.7,
                                explode=explode, colors=colors, shadow=False, startangle=39, rotatelabels =True, textprops={'fontsize': 12})

for label, pct_text in zip(labels, pct_texts):
    pct_text.set_rotation(label.get_rotation())

for i, (label, pct_text) in enumerate(zip(labels, pct_texts)):
    label.set_rotation(0)
    if i in [1,2]:
        pct_text.set_color('white')

ax1.axis('equal')

plt.axis("equal")
plt.tight_layout()
fig1.savefig('07_Pie_temperature_intervals.svg', pad_inches=0, transparent=True, bbox_inches='tight', dpi=250)

Percentage of the tiles (with at least 1 building) inside temperature intervals<br>
- 26.2% tiles T < 21°C <br>
- 68.8% tiles 25°C >= T >= 21°C <br>
-  5.0% tiles T > 25°C

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_axis_off()

gdf_grid[gdf_grid['Temperature']>25].plot(ax=ax, markersize=10, alpha=0.8,  color=sns.color_palette('plasma')[5])
gdf_grid[(gdf_grid['Temperature']<=25) & (gdf_grid['Temperature'] >= 21)].plot(ax=ax, markersize=10, alpha=0.8, color=sns.color_palette('plasma')[2])
gdf_grid[gdf_grid['Temperature']<21].plot(ax=ax, markersize=10, alpha=0.8, color=sns.color_palette('plasma')[0])

ctx.add_basemap(ax, crs=gdf_st_clean.to_crs(epsg=3857).crs.to_string(), source=ctx.providers.Stamen.Terrain)
plt.tight_layout()
fig.savefig('08_Surafe_Temperature_grid_intervals.png', bbox_inches='tight', dpi=250)

### Buildings in tiles with temperature below 21°C

In [ ]:
df_below21 = pd.Series(df_bh_grid[df_bh_grid["Temperature"] < 21 ]['b_area_ratio'].sum(axis=0) * 100 / df_bh_grid[df_bh_grid["Temperature"] < 21 ]['b_area_ratio'].sum(axis=0).sum(axis=0), name="Occupy").to_frame()
df_below21 = df_below21[df_below21.values>2].sort_values(by='Occupy', ascending=False)
df_below21

Note: sum of area percentage in "Occupy" column is not 100% because buildings with area < 2% have been dropped

Average height calculated from weighted mean height values

In [ ]:
avg_height_below21 = gdf_grid_extended.loc[df_bh_grid[df_bh_grid["Temperature"] < 21 ].index, "WM_Height"].sum() / len(df_bh_grid[df_bh_grid["Temperature"] < 21 ].index)
print("Average height of tiles with temperature below 21°C is %.2f m." %avg_height_below21)

In [ ]:
labels = df_below21.index.values
values = df_below21.values.flatten()
explode = np.size(df_below21.values)*(0.02,)
explode = list(explode)
explode[0] = 0.05
explode = tuple(explode)

colors = plt.cm.plasma(np.linspace(0.12,0.28,np.size(df_below21.values)))
fig1, ax1 = plt.subplots(figsize=(8,5))
patches, labels, pct_texts = ax1.pie(values, labels=labels, labeldistance=1.05, autopct='%1.1f%%', pctdistance=0.8,
                                explode=explode, colors=colors, shadow=False, startangle=39, rotatelabels =True, textprops={'fontsize': 12})

for label, pct_text in zip(labels, pct_texts):
    pct_text.set_rotation(label.get_rotation())
    pct_text.set_color('white')

for label, pct_text in zip(labels, pct_texts):
    label.set_rotation(0)

ax1.axis('equal')
ax1.set_title("Plochy budov v oblasti s teplotou pod 21°C", fontsize=17, color=colors[1], fontweight="bold", x=0.6)
plt.axis("equal")
plt.tight_layout()
fig1.savefig('09_Pie_below21.svg', transparent=True, bbox_inches='tight')

Area ratio of dominant buildings in the tiles with temperature below 21°C <br>
- 46.1% stavba pro rodinnou rekreaci
- 23.0% rodinny dum

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()
gdf_bh[gdf_bh['zpusob_vyu'] == 'stavba pro rodinnou rekreaci'].centroid.plot(ax=ax, markersize=10, alpha=0.5, color=colors[0])

ctx.add_basemap(ax, crs=gdf_bh.crs.to_string(), source=ctx.providers.Stamen.Terrain)
plt.tight_layout()
fig.savefig('10_Below21_buildings_map.png', bbox_inches='tight', dpi=250)

Location of buildings "stavba pro rodinnou rekreaci" is mainly on the outskirts and remote places of Brno.

In [ ]:
buildings_below21_cnt = df_bh_grid[df_bh_grid["Temperature"] < 21 ]['cnt'].sum(axis=0).sort_values(ascending=True)[-18:]

fig, ax = plt.subplots(1, 1, figsize=(9, 10))
ax.set_xlabel('')
color = plt.cm.plasma(np.linspace(0.32,0.10,18))
buildings_below21_cnt.plot(kind='barh', ax=ax, zorder=2, color=color)
ax.grid(axis='x', linestyle='--', linewidth="0.8", color="k", alpha=0.3, zorder=0)

ax.spines[['top','right','bottom', 'left']].set_visible(False)
ax.yaxis.set_ticks_position('none')
ax.xaxis.set_ticks_position('none')
ax.set_ylabel(None)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=14)
xlabels = ax.get_xticks().astype('i')[1:-1]
ax.set_xticks(xlabels)
ax.set_xticklabels(xlabels, fontsize=14)
ax.tick_params(axis='y', pad=12)

for i, v in enumerate(buildings_below21_cnt.values):
    plt.text(v+25, i, str(int(v)), color=color[i], va="center", fontsize=14)

fig.savefig('11_Barh_below21.svg', bbox_inches='tight')

Absolute count of individual buildings inside the tiles with temperature below 21°C

### Buildings in tiles with temperature between 21°C÷25°C

In [ ]:
df_between = pd.Series(df_bh_grid[(df_bh_grid["Temperature"] <= 25) & (df_bh_grid["Temperature"] >= 21)]['b_area_ratio'].sum(axis=0) * 100 / df_bh_grid[(df_bh_grid["Temperature"] <= 25) & (df_bh_grid["Temperature"] >= 21)]['b_area_ratio'].sum(axis=0).sum(axis=0), name="Occupy").to_frame()
df_between = df_between[df_between.values>2].sort_values(by='Occupy', ascending=False)
df_between

Note: sum of area percentage in "Occupy" column is not 100% because buildings with area < 2% have been dropped

Average height calculated from weighted mean height values

In [ ]:
avg_height_between = gdf_grid_extended.loc[df_bh_grid[(df_bh_grid["Temperature"] <= 25) & (df_bh_grid["Temperature"] >= 21)].index, "WM_Height"].sum()/len(df_bh_grid[(df_bh_grid["Temperature"] <= 25) & (df_bh_grid["Temperature"] >= 21)].index)
print("Average height of tiles with temperature between 21°C÷25°C is %.2f m." %avg_height_between)

In [ ]:
labels = df_between.index.values
values = df_between.values.flatten()
explode = np.size(df_between.values)*(0.01,)
explode = list(explode)
explode[0] = 0.08
explode[1] = 0.08
explode[6] = 0.12
explode = tuple(explode)

colors = plt.cm.plasma(np.linspace(0.5,0.2,np.size(df_between.values)))
fig1, ax1 = plt.subplots(figsize=(8,5))
patches, labels, pct_texts = ax1.pie(values, labels=labels, labeldistance=1.05, autopct='%1.1f%%', pctdistance=0.8,
                                explode=explode, colors=colors, shadow=False, startangle=20, rotatelabels =True, textprops={'fontsize': 12})

for label, pct_text in zip(labels, pct_texts):
    pct_text.set_rotation(label.get_rotation())

for label, pct_text in zip(labels, pct_texts):
    label.set_rotation(0)
    pct_text.set_color('white')

ax1.axis('equal')
ax1.set_title("Plochy budov v oblasti s teplotou medzi 21°C÷25°C", fontsize=17, color=colors[1], fontweight="bold", x=0.55)
plt.axis("equal")
plt.tight_layout()
fig1.savefig('12_Pie_between.svg', bbox_inches='tight')

Area ratio of dominant buildings in the tiles with temperature between 21°C÷25°C <br>
- 23.6% rodinny dum
- 15.7% bytovy dum
- 11.5% jina stavba
- 10.6% stavba obcanskeho vybaveni

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()
gdf_bh[gdf_bh['zpusob_vyu'] == 'rodinný dům'].centroid.plot(ax=ax, markersize=10, alpha=0.4, color=colors[0])
gdf_bh[gdf_bh['zpusob_vyu'] == 'bytový dům'].centroid.plot(ax=ax, markersize=10, alpha=0.4, color=colors[1])
gdf_bh[gdf_bh['zpusob_vyu'] == 'objekt k bydlení'].centroid.plot(ax=ax, markersize=10, alpha=0.4, color=colors[6])

ctx.add_basemap(ax, crs=gdf_bh.crs.to_string(), source=ctx.providers.Stamen.Terrain)
plt.tight_layout()
fig.savefig('13_Between_buildings_map.png', bbox_inches='tight', dpi=250)

Buildings "rodinny dum", "bytovy dum" and "objekt k bydleni" are located in the center of Brno, as well as on the outskirts.

In [ ]:
buildings_between_cnt = df_bh_grid[(df_bh_grid["Temperature"] <= 25) & (df_bh_grid["Temperature"] >= 21)]['cnt'].sum(axis=0).sort_values(ascending=True)[-17:]

fig, ax = plt.subplots(1, 1, figsize=(9, 10))
ax.set_xlabel('')
color = plt.cm.plasma(np.linspace(0.22,0.5,17))
buildings_between_cnt.plot(kind='barh', ax=ax, zorder=2, color=color)
ax.grid(axis='x', linestyle='--', linewidth="0.8", color="k", alpha=0.2, zorder=0)

ax.spines[['top','right','bottom', 'left']].set_visible(False)
ax.yaxis.set_ticks_position('none')
ax.xaxis.set_ticks_position('none')
ax.set_ylabel(None)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=14)
xlabels = ax.get_xticks().astype('i')[1:-1]
ax.set_xticks(xlabels)
ax.set_xticklabels(xlabels, fontsize=14)
ax.tick_params(axis='y', pad=12)

for i, v in enumerate(buildings_between_cnt.values):
    plt.text(v+200, i, str(int(v)), color=color[i], va="center", fontsize=14)

fig.savefig('14_Barh_between.svg', bbox_inches='tight')

Absolute count of individual buildings inside the tiles with temperature between 21°C÷25°C

### Buildings in tiles with temperature above 25°C

In [ ]:
df_above25 = pd.Series(df_bh_grid[df_bh_grid["Temperature"] > 25 ]['b_area_ratio'].sum(axis=0) * 100 / df_bh_grid[df_bh_grid["Temperature"] > 25 ]['b_area_ratio'].sum(axis=0).sum(axis=0), name="Occupy").to_frame()
df_above25 = df_above25[df_above25.values>2].sort_values(by='Occupy', ascending=False)
df_above25

Note: sum of area percentage in "Occupy" column is not 100% because buildings with area < 2% have been dropped

Average height calculated from weighted mean height values

In [ ]:
avg_height_above25 = gdf_grid_extended.loc[df_bh_grid[df_bh_grid["Temperature"] > 25].index, "WM_Height"].sum() / len(df_bh_grid[df_bh_grid["Temperature"] > 25 ].index)
print("Average height of tiles with temperature above 25°C is %.2f m." %avg_height_above25)

In [ ]:
labels = df_above25.index.values
values = df_above25.values.flatten()
explode = np.size(df_above25.values)*(0.02,)
explode = list(explode)
explode[0] = 0.08
explode[3] = 0.12
explode = tuple(explode)

colors = plt.cm.plasma(np.linspace(0.9,0.75,np.size(df_above25.values)))
fig1, ax1 = plt.subplots(figsize=(8,5))
patches, labels, pct_texts = ax1.pie(values, labels=labels, labeldistance=1.05, autopct='%1.1f%%', pctdistance=0.8,
                                explode=explode, colors=colors, shadow=False, startangle=45, rotatelabels =True, textprops={'fontsize': 12})

for label, pct_text in zip(labels, pct_texts):
    pct_text.set_rotation(label.get_rotation())

for label, pct_text in zip(labels, pct_texts):
    label.set_rotation(0)

ax1.axis('equal')
ax1.set_title("Plochy budov v oblasti s teplotou nad 25°C", fontsize=17, color=colors[1], fontweight="bold", x=0.52)
plt.axis("equal")
plt.tight_layout()
fig1.savefig('15_Pie_above25.svg', bbox_inches='tight')

Area ratio of dominant buildings in the tiles with temperature above 25°C <br>
- 50.9% stavba pro vyrobu a skladovani
- 12.0% stavba obcanskeho vybaveni
- 11.5% jina stavba

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.set_axis_off()
gdf_bh[gdf_bh['zpusob_vyu'] == 'stavba pro výrobu a skladování'].centroid.plot(ax=ax, markersize=15, alpha=0.6, color=colors[0])
gdf_bh[gdf_bh['zpusob_vyu'] == 'průmyslový objekt'].centroid.plot(ax=ax, markersize=15, alpha=0.6, color=colors[3])

ctx.add_basemap(ax, crs=gdf_bh.crs.to_string(), source=ctx.providers.Stamen.Terrain)
plt.tight_layout()
fig.savefig('16_Above25_buildings_map.png', bbox_inches='tight', dpi=250)

Industrial buildings "stavba pro vyrobu a skladovani" and "prumyslovy objekt" are mainly located in the east of the Brno.

In [ ]:
buildings_above25_cnt = df_bh_grid[df_bh_grid["Temperature"] > 25]['cnt'].sum(axis=0).sort_values(ascending=True)[-17:]

fig, ax = plt.subplots(1, 1, figsize=(9, 10))
ax.set_xlabel('')
color = plt.cm.plasma(np.linspace(0.7,0.88,17))
buildings_above25_cnt.plot(kind='barh', ax=ax, zorder=2, color=color)
ax.grid(axis='x', linestyle='--', linewidth="0.8", color="k", alpha=0.2, zorder=0)

ax.spines[['top','right','bottom', 'left']].set_visible(False)
ax.yaxis.set_ticks_position('none')
ax.xaxis.set_ticks_position('none')
ax.set_ylabel(None)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=14)
xlabels = ax.get_xticks().astype('i')[1:-2]
ax.set_xticks(xlabels)
ax.set_xticklabels(xlabels, fontsize=14)
ax.tick_params(axis='y', pad=12)

for i, v in enumerate(buildings_above25_cnt.values):
    plt.text(v+10, i, str(int(v)), color=color[i], va="center", fontsize=14)

fig.savefig('17_Barh_above25.svg', bbox_inches='tight')

Absolute count of individual buildings inside the tiles with temperature above 25°C